# LGBM regressor
LGM regressor scorer ofte bra, jeg har valgt å bruke denne for å se hva vi kan få ut av datasettet uten større mengde feature engineering.

## importere bibloteker

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.impute import SimpleImputer


## for å hente ut data

In [2]:
def load_data():
    # Read training, test, and sample submission datasets
    train = pd.read_csv("input/train.csv")
    test = pd.read_csv("input/test.csv")
    sample_submission = pd.read_csv("input/sample_submission.csv")

    # Return all three datasets
    return train, test, sample_submission


## Evt. feature engineering

In [3]:
def create_engineered_features(df):
    """
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with raw features.

    Returns
    -------
    pandas.DataFrame
        DataFrame with added engineered features (non-destructive copy).
    """
    df = df.copy()

    # Feature 1: speed / curvature (protect against div-by-zero with small epsilon)
    speed_o_curve = df["speed_limit"] / (df["curvature"] + 1e-6)
    df["speed_o_curve"] = speed_o_curve.fillna(0)

    # Feature 2 (disabled): speed squared
    # speed_x_speed = df["speed_limit"] ** 2
    # df["speed_x_speed"] = speed_x_speed.fillna(0)

    # Feature 3 (disabled): speed * reported accidents
    # speed_x_accidents = df["speed_limit"] * df["num_reported_accidents"]
    # df["speed_x_accidents"] = speed_x_accidents.fillna(0)

    # Feature 4: visibility risk components (lighting + weather) and composite
    lighting_w = {"night": 0.9, "dim": 0.3, "daylight": 0.1}
    weather_w = {"foggy": 0.8, "rainy": 0.7, "clear": 0.1}

    df["lighting_risk"] = df["lighting"].map(lighting_w).fillna(0)
    df["weather_risk"] = df["weather"].map(weather_w).fillna(0)
    df["visibility_composite"] = (df["lighting_risk"] + df["weather_risk"]) / 2

    # Feature 5: time of day as ordinal integer
    time_order = {"morning": 1, "evening": 2, "afternoon": 3}
    df["time_as_int"] = df["time_of_day"].map(time_order)

    # # Feature 6: log1p(speed / curvature)
    # df["log_speed_o_curve"] = np.log1p(speed_o_curve)
    #
    # # Feature 7: curvature * speed
    # df["curvature_x_speed"] = df["curvature"] * df["speed_limit"]
    #
    # # Feature 8: reported accidents per lane (add 1 to avoid div-by-zero)
    # df["accidents_o_lanes"] = df["num_reported_accidents"] / (df["num_lanes"] + 1)
    #
    # # Feature 9: speed * time (ordinal)
    # df["speed_time_interaction"] = df["speed_limit"] * df["time_as_int"]
    #
    # # Feature 10: curvature * time (ordinal)
    # df["curvature_time_interaction"] = df["curvature"] * df["time_as_int"]

    return df


## preparere features for bruk i modell

In [4]:
def prepare_features(train, test):
    # Create engineered features
    train = create_engineered_features(train)
    test = create_engineered_features(test)

    # Separate features and target variable
    X = train.drop(columns=["accident_risk", "id"])
    y = train["accident_risk"]


    # Define categorical feature names
    categorical_features = [
        "road_type",
        "lighting",
        "weather",
        "time_of_day"
    ]

    # Define numerical feature names
    numerical_features = [
        "num_lanes",                # base
        "curvature",                # base
        "speed_limit",              # base
        "num_reported_accidents",   # base

        "speed_o_curve",            # Feature 1
        # "speed_x_speed",          # Feature 2 (disabled)
        # "speed_x_accidents",      # Feature 3 (disabled)

        "visibility_composite",     # Feature 4 (composite)
        "lighting_risk",            # Feature 4 (component)
        "weather_risk",             # Feature 4 (component)

        "time_as_int",              # Feature 5
        # "log_speed_o_curve",        # Feature 6
        # "curvature_x_speed",        # Feature 7
        # "accidents_o_lanes",        # Feature 8
        # "speed_time_interaction",   # Feature 9
        # "curvature_time_interaction",  # Feature 10
    ]


    boolean_features = [
        "holiday",
        "school_season",
        "road_signs_present",
        "public_road"
    ]

    print(f"Features: {X.shape[1]}")

        # --- Normalize column dtypes to avoid np.isnan / pd.NA type issues ---
    for col in categorical_features:
        if col in X.columns:
            X[col] = X[col].astype("string").fillna("__MISSING__")
            test[col] = test[col].astype("string").fillna("__MISSING__")


    for col in boolean_features:
        X[col] = X[col].astype("boolean")
        test[col] = test[col].astype("boolean")

    for col in numerical_features:
        # Coerce nullable numerics (like Int64) to float64
        X[col] = pd.to_numeric(X[col], errors="coerce")
        test[col] = pd.to_numeric(test[col], errors="coerce")

    # --- Pipelines for each type ---
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ])

    numerical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        # with_mean=False keeps it compatible with sparse output
        ("scaler", StandardScaler(with_mean=False))
    ])

    # Combine everything in a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            #("cat", categorical_transformer, categorical_features),
            ("num", numerical_transformer, numerical_features),
            # You can include boolean features as numeric 0/1
            ("bool", "passthrough", boolean_features)
        ],
        sparse_threshold=0.3
    )

    # Return features, target, test set, and preprocessor
    return X, y, test, preprocessor

## Hyperparamterisation med optina og LGBM

# 1, 2, 3

In [67]:
# pip install optuna lightgbm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMRegressor
import optuna
import numpy as np

def optuna_lgbm_search(X, y, preprocessor, n_trials=50, timeout=None, random_state=42, verbose = True):
    """
    Run Optuna hyperparameter search for a LightGBM regressor inside a preprocessing pipeline.

    Parameters
    ----------
    X : array-like or DataFrame
        Features.
    y : array-like
        Target.
    preprocessor : transformer
        Any sklearn-compatible transformer (e.g., ColumnTransformer).
    n_trials : int
        Number of Optuna trials.
    timeout : int or None
        Stop after this many seconds (optional).
    random_state : int
        Random seed.

    Returns
    -------
    best_model : Pipeline
        Fitted pipeline with the best-found hyperparameters.
    study : optuna.study.Study
        The Optuna study object (for plots/inspection).
    best_params : dict
        Best hyperparameters found by Optuna.
    """

    if verbose:
        optuna.logging.set_verbosity(optuna.logging.INFO)
    else:
        optuna.logging.set_verbosity(optuna.logging.WARNING)


    def objective(trial: optuna.Trial) -> float:
        params = {
            # Core LGBM hyperparameters to search
            "n_estimators": trial.suggest_int("n_estimators", 200, 2000),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "num_leaves": trial.suggest_int("num_leaves", 16, 512, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 1.0, log=True),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
            "subsample_freq": trial.suggest_int("subsample_freq", 0, 10),
        }

        # Build the pipeline for this trial
        model = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("regressor", LGBMRegressor(
                **params,
                random_state=random_state,
                n_jobs=-1,
                verbosity=-1
            ))
        ])

        # 5-fold CV RMSE (negated by sklearn, so we negate back)
        cv = KFold(n_splits=6, shuffle=True, random_state=random_state)
        scores = cross_val_score(
            model, X, y,
            scoring="neg_root_mean_squared_error",
            cv=cv,
            n_jobs=-1
        )
        rmse = -np.mean(scores)
        return rmse

    def print_callback(study: optuna.Study, trial: optuna.trial.FrozenTrial):
        if verbose:
            print(
                f"[Trial {trial.number}] value={trial.value:.5f} | "
                #f"best={study.best_value:.5f} | params={trial.params}"
            )

    study = optuna.create_study(direction="minimize")
    study.optimize(
        objective,
        n_trials=n_trials,
        timeout=timeout,
        callbacks=[print_callback] if verbose else None,
        show_progress_bar=verbose
    )

    best_params = study.best_params.copy()

    # Build and fit the best model on full data
    best_model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", LGBMRegressor(
            **best_params,
            random_state=random_state,
            n_jobs=-1,
            verbosity=-1
        ))
    ])
    best_model.fit(X, y)

    return best_model, study, best_params


In [68]:
from sklearn import clone
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler
from joblib import Memory


def optuna_lgbm_search_fast(
    X, y, preprocessor, n_trials=50, timeout=None, random_state=42, verbose=True,
    n_splits=5, early_stopping_rounds=100, first_stage_trials=20
):
    # Sampler + pruner
    sampler = TPESampler(seed=random_state, multivariate=True, group=True, n_startup_trials=10)
    pruner = SuccessiveHalvingPruner(min_resource=5, reduction_factor=3)

    if verbose:
        optuna.logging.set_verbosity(optuna.logging.INFO)
    else:
        optuna.logging.set_verbosity(optuna.logging.WARNING)

    # Cache preprocessing
    memory = Memory(location=".cache_optuna_lgbm", verbose=0)

    # Prior good params (center searches)
    prior = {
        "n_estimators": 525, "learning_rate": 0.06, "max_depth": 8, "num_leaves": 64,
        "subsample": 0.8, "colsample_bytree": 0.9, "reg_lambda": 0.6, "reg_alpha": 0.2
    }

    def suggest_params(trial):
        # Fixed distributions (constant across all trials)
        lr = trial.suggest_float("learning_rate", 1e-3, 0.3, log=True)
        n_est = trial.suggest_int("n_estimators", 200, 2000)
        max_depth = trial.suggest_int("max_depth", 3, 12)

        # sample raw leaves with constant distribution; clamp to 2**max_depth later
        num_leaves_raw = trial.suggest_int("num_leaves_raw", 2, 512, log=True)
        max_leaves = 2 ** max_depth if max_depth > 0 else 2
        num_leaves = int(min(num_leaves_raw, max_leaves))

        subsample = trial.suggest_float("subsample", 0.5, 1.0)
        subsample_freq = trial.suggest_int("subsample_freq", 0, 10)
        colsample = trial.suggest_float("colsample_bytree", 0.5, 1.0)

        reg_lambda = trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True)
        reg_alpha  = trial.suggest_float("reg_alpha", 1e-3, 1.0, log=True)

        min_child_samples = trial.suggest_int("min_child_samples", 5, 100)
        min_split_gain   = trial.suggest_float("min_split_gain", 0.0, 1.0)
        min_child_weight = trial.suggest_float("min_child_weight", 1e-3, 10.0, log=True)

        # Normalize conditionals AFTER sampling
        if subsample >= 0.999:
            subsample = 1.0
            subsample_freq = 0

        return {
            "n_estimators": n_est,
            "learning_rate": lr,
            "max_depth": max_depth,
            "num_leaves": num_leaves,
            "subsample": subsample,
            "subsample_freq": subsample_freq,
            "colsample_bytree": colsample,
            "reg_lambda": reg_lambda,
            "reg_alpha": reg_alpha,
            "min_child_samples": min_child_samples,
            "min_split_gain": min_split_gain,
            "min_child_weight": min_child_weight,
        }


    def objective(trial: optuna.Trial) -> float:
        params = suggest_params(trial)

        # keep conditional subsampling sane
        if params["subsample"] >= 0.999:
            params["subsample"] = 1.0
            params["subsample_freq"] = 0

        # NOTE: we won't use a Pipeline here inside CV because we must transform eval_set
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

        rmses = []
        for fold, (trn_idx, val_idx) in enumerate(cv.split(X)):
            # Slice
            if hasattr(X, "iloc"):
                X_tr, X_va = X.iloc[trn_idx], X.iloc[val_idx]
                y_tr, y_va = y.iloc[trn_idx], y.iloc[val_idx]
            else:
                X_tr, X_va = X[trn_idx], X[val_idx]
                y_tr, y_va = y[trn_idx], y[val_idx]

            # Fit a fresh copy of the preprocessor on the train fold, transform both
            prep = clone(preprocessor)
            Xtr_t = prep.fit_transform(X_tr, y_tr)
            Xva_t = prep.transform(X_va)

            # Regressor with controlled parallelism to avoid oversubscription
            reg = LGBMRegressor(
                **params,
                random_state=random_state,
                n_jobs=1,
                verbosity=-1
            )

            # Early stopping via callback (works across LightGBM versions)
            reg.fit(
                Xtr_t, y_tr,
                eval_set=[(Xva_t, y_va)],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=False)]
            )

            y_pred = reg.predict(Xva_t, num_iteration=reg.best_iteration_)
            rmse = float(np.sqrt(np.mean((y_pred - y_va) ** 2)))
            rmses.append(rmse)

            trial.report(np.mean(rmses), step=fold + 1)
            if trial.should_prune():
                raise optuna.TrialPruned()

        return float(np.mean(rmses))

    # Create study first
    study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)

    # Enqueue your prior best so it is evaluated as an actual trial
    prior_params = {
        "n_estimators": 525, "learning_rate": 0.06, "max_depth": 8, "num_leaves": 64,
        "subsample": 0.8, "subsample_freq": 1, "colsample_bytree": 0.9,
        "reg_lambda": 0.6, "reg_alpha": 0.2, "min_child_samples": 20,
        "min_split_gain": 0.0, "min_child_weight": 1.0
    }
    study.enqueue_trial(prior_params)

    # Then optimize
    study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=verbose)


    best_params = study.best_params.copy()

    # Fit best on full data (respect best_iteration with a small buffer of n_estimators)
    best_model = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("regressor", LGBMRegressor(
                **best_params,
                random_state=random_state,
                n_jobs=-1,  # fast final fit
                verbosity=-1
            ))
        ],
        memory=memory
    )
    # One last early stopping pass on a small validation split (optional), or just fit:
    best_model.fit(X, y)

    return best_model, study, best_params


In [69]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import warnings


# pip install optuna lightgbm


def optuna_lgbm_search3(
    X, y, preprocessor, n_trials=100, timeout=None, random_state=42, verbose=True, n_splits=5, n_bins=10
):
    """
    Optuna search for LightGBM with:
      - Early stopping inside CV (no direct n_estimators tuning)
      - Pruning (Hyperband by default)
      - StratifiedKFold for regression via binned y
      - Leakage-safe preprocessing per fold
    Returns (best_model, study, best_params)
    """
    # Logging
    optuna.logging.set_verbosity(optuna.logging.INFO if verbose else optuna.logging.WARNING)

    # Pruner & Sampler
    sampler = optuna.samplers.TPESampler(seed=random_state, multivariate=True, group=True)
    pruner = optuna.pruners.HyperbandPruner(reduction_factor=4)

    # Bin y for stratified regression CV
    y = np.asarray(y)
    # handle constant or tiny datasets gracefully
    if len(np.unique(y)) < n_bins:
        bins = min(len(np.unique(y)), max(2, len(np.unique(y))))
    else:
        bins = n_bins
    # np.digitize needs bin edges:
    quantiles = np.linspace(0, 1, bins + 1)
    edges = np.unique(np.quantile(y, quantiles))
    if len(edges) < 3:  # fallback to plain KFold if pathological
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        y_strat = np.digitize(y, np.linspace(y.min(), y.max(), n_splits + 1)[1:-1], right=True)
    else:
        y_strat = np.digitize(y, edges[1:-1], right=True)
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    def objective(trial: optuna.Trial) -> float:
        # Search space
        boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "goss"])
        params = {
            "boosting_type": boosting_type,
            # keep n_estimators large; we rely on early stopping per fold
            "n_estimators": 20000,
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_categorical("max_depth", [-1, 3, 4, 5, 6, 7, 8, 10, 12, 14, 16]),
            "num_leaves": trial.suggest_int("num_leaves", 16, 2048, log=True),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 300),
            "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0) if boosting_type == "gbdt" else 1.0,
            "subsample_freq": trial.suggest_int("subsample_freq", 1, 10) if boosting_type == "gbdt" else 0,
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 200.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 10.0, log=True),
            "max_bin": trial.suggest_int("max_bin", 63, 511),
            # misc
            "random_state": random_state,
            "n_jobs": -1,
            "verbosity": -1,
        }

        fold_rmses = []
        best_iters = []

        # CV with leakage-safe preprocessing + early stopping
        for fold_idx, (tr_idx, va_idx) in enumerate(skf.split(X, y_strat), 1):
            X_tr, X_va = X.iloc[tr_idx] if hasattr(X, "iloc") else X[tr_idx], X.iloc[va_idx] if hasattr(X, "iloc") else X[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]

            # Fit preprocessor on train only, transform both
            fold_pre = clone(preprocessor)
            X_tr_t = fold_pre.fit_transform(X_tr, y_tr)
            X_va_t = fold_pre.transform(X_va)

            # Build model
            model = LGBMRegressor(**params)

            # Early stopping callback + pruning bridge
            callbacks = [
                lgb.early_stopping(stopping_rounds=200, verbose=False),
                # Report intermediate metrics to Optuna for pruning
                lgb.log_evaluation(period=0),  # keep silent
            ]

            # Fit with eval_set for early stopping
            model.fit(
                X_tr_t, y_tr,
                eval_set=[(X_va_t, y_va)],
                eval_metric="rmse",
                callbacks=callbacks,
            )

            # Predict with best_iteration_
            y_pred = model.predict(X_va_t, num_iteration=model.best_iteration_)
            rmse = mean_squared_error(y_va, y_pred) ** 0.5
            fold_rmses.append(rmse)
            best_iters.append(model.best_iteration_)

            # Let Optuna see progress for pruning
            trial.report(np.mean(fold_rmses), fold_idx)
            if trial.should_prune():
                raise optuna.TrialPruned()

        score = float(np.mean(fold_rmses))
        # store a hint for final n_estimators
        trial.set_user_attr("avg_best_iteration", int(np.median(best_iters)))
        return score

    study = optuna.create_study(
        direction="minimize",
        sampler=sampler,
        pruner=pruner
    )
    study.optimize(
        objective,
        n_trials=n_trials,
        timeout=timeout,
        show_progress_bar=verbose,
        gc_after_trial=True,
    )

    best_params = study.best_params.copy()
    # Use a robust central tendency for best iteration across folds
    best_iters = [t.user_attrs.get("avg_best_iteration") for t in study.trials if t.user_attrs.get("avg_best_iteration")]
    if len(best_iters):
        final_n_estimators = int(np.median(best_iters))
    else:
        final_n_estimators = 1000  # fallback

    # Build final pipeline (fit on full data)
    final_params = {
        **best_params,
        "n_estimators": max(50, final_n_estimators),  # guard
        "random_state": random_state,
        "n_jobs": -1,
        "verbosity": -1,
    }

    best_model = Pipeline(steps=[
        ("preprocessor", clone(preprocessor)),
        ("regressor", LGBMRegressor(**final_params))
    ])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        best_model.fit(X, y)

    return best_model, study, best_params


## skape submission fil

In [79]:
def generate_submission(lgbm_model, xgb_model, X, y, test, sample_submission):
    # Preprocess features
    X_processed = lgbm_model.named_steps["preprocessor"].fit_transform(X)
    test_processed = lgbm_model.named_steps["preprocessor"].transform(test)

    # Train LightGBM
    lgbm_model.named_steps["regressor"].fit(
        X_processed,
        y,
        eval_set=[(X_processed, y)],
        eval_metric="rmse",
        callbacks=[
            early_stopping(stopping_rounds=50),
            log_evaluation(period=50)
        ]
    )


    # Generate predictions from both models
    preds_lgbm = lgbm_model.named_steps["regressor"].predict(test_processed)


    # Prepare submission file
    submission = sample_submission.copy()
    # submission["accident_risk"] = final_predictions
    submission["accident_risk"] = preds_lgbm

    # Save CSV for Kaggle submission
    submission.to_csv("submissions/hyperparameter.csv", index=False)


    print("Submission file saved in submissions folder.")

In [75]:
def main():
    # Load datasets
    train, test, sample_submission = load_data()

    # Prepare features and preprocessing
    X, y, test, preprocessor = prepare_features(train, test)

    # Build both models
    #lgbm_model = build_lgbm_model(preprocessor)

    # previous hyperparameter search (better)
    best_model, study, best_params = optuna_lgbm_search3(
        X, y, preprocessor,
        n_trials=500,   # adjust as you like
        timeout=None,   # or set a number of seconds
        random_state=42
    )

    # new hyperparameter search with fast version of optuna (worse)
    #best_model, study, best_params = optuna_lgbm_search_fast(X, y, preprocessor)

    print(f"Best hyperparameters: {best_params}")
    print(f"Best RMSE: {study.best_value:.2f}")
    print(f"Best trial: {study.best_trial}")
    #xgb_model = build_xgb_model(preprocessor)

    # Generate final averaged submission
    generate_submission(best_model, _, X, y, test, sample_submission)

In [76]:
main()

C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``group`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-10-30 00:58:06,884] A new study created in memory with name: no-name-7623c4a1-21ae-4a96-935e-0b85e35c251d


Features: 17


  0%|          | 0/500 [00:00<?, ?it/s]C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\

[I 2025-10-30 00:58:32,041] Trial 0 finished with value: 0.05658194676904295 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.06504856968981275, 'max_depth': 12, 'num_leaves': 38, 'min_child_samples': 59, 'min_split_gain': 0.3042422429595377, 'colsample_bytree': 0.762378215816119, 'reg_lambda': 0.052686399294954014, 'reg_alpha': 0.0028585493941961923, 'max_bin': 337}. Best is trial 0 with value: 0.05658194676904295.


Best trial: 0. Best value: 0.0565819:   0%|          | 1/500 [00:25<3:32:12, 25.52s/it]C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitt

...

[I 2025-10-30 11:18:55,536] Trial 499 pruned. 


Best trial: 404. Best value: 0.0560104: 100%|██████████| 500/500 [10:20:48<00:00, 74.50s/it] 


Best hyperparameters: {'boosting_type': 'goss', 'learning_rate': 0.014614918918675512, 'max_depth': 14, 'num_leaves': 310, 'min_child_samples': 13, 'min_split_gain': 0.022511986213364955, 'colsample_bytree': 0.8652754873375939, 'reg_lambda': 1.9628584361818673, 'reg_alpha': 0.0004203455868142346, 'max_bin': 168}
Best RMSE: 0.06
Best trial: FrozenTrial(number=404, state=1, values=[0.056010408418806], datetime_start=datetime.datetime(2025, 10, 30, 9, 29, 33, 753995), datetime_complete=datetime.datetime(2025, 10, 30, 9, 30, 42, 169369), params={'boosting_type': 'goss', 'learning_rate': 0.014614918918675512, 'max_depth': 14, 'num_leaves': 310, 'min_child_samples': 13, 'min_split_gain': 0.022511986213364955, 'colsample_bytree': 0.8652754873375939, 'reg_lambda': 1.9628584361818673, 'reg_alpha': 0.0004203455868142346, 'max_bin': 168}, user_attrs={'avg_best_iteration': 2083}, system_attrs={'completed_rung_0': 0.05600338064299917}, intermediate_values={1: 0.05607485076839703, 2: 0.0559802997868

C:\Users\tebje\Documents\HVLIT\GithubRepos\AT-Road-Accident-Risk\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Submission file saved in submissions folder.
